# Understanding retries
Here description of what this does.

In [1]:
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
import re
import statistics

import numpy as np
import pandas as pd

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)
jobs = []

boincsites = ('ANALY_BOINC',
              'BOINC',
              'BOINC-ES',
              'BOINC-ES_CLONE',
              'BOINC_CHECKPOINT',
              'BOINC_MCORE',
              'BOINC_VIRTUAL')

cloudsites = ('ANALY_IAAS',
              'ANALY_IAAS_TEST',
              'CERN-EXTENSION_CERN',
              'CERN-EXTENSION_DE',
              'CERN-EXTENSION_HARVESTER',
              'CERN-EXTENSION_TEST',
              'CERN-EXTENSION_UK',
              'CERN-P1',
              'CERN-P1_DYNAMIC_MCORE',
              'CERN-P1_DYNAMIC_MCORE_LOWMEM',
              'CERN-P1_DYNAMIC_SCORE',
              'HELIX_NEBULA_ATOS',
              'HELIX_NEBULA_EGI',
              'IAAS',
              'IAAS_MCORE',
              'IAAS_MCORE_HIMEM',
              'IN2P3-CC-T3_VM01',
              'IN2P3-CC-T3_VM01_SL7',
              'IN2P3-CC-T3_VM02',
              'IN2P3-CC-T3_VM02_SL7',
              'RAL-AZURE_VAC',
              'RAL-LCG2_VAC',
              'UIO_CLOUD',
              'UKI-LT2-UCL-HEP_VAC',
              'UKI-NORTHGRID-MAN-HEP_MCOREVAC',
              'UKI-SCOTGRID-GLASGOW_MCOREVAC',
              'UKI-SOUTHGRID-BHAM-HEP_MCOREVAC',
              'UKI-SOUTHGRID-OX-HEP_CLOUD',
              'UNIBE-LHEP_CLOUD',
              'UNIBE-LHEP_CLOUD_MCORE')

hpcsites = ('ALCF_Cooley',
            'ALCF_Theta',
            'ALCF_Theta_ES',
            'ANALY_CSCS-HPC',
            'ANALY_HPC2N',
            'ANALY_NSC',
            'ANALY_ORNL_Titan',
            'ANALY_RRC-KI-HPC',
            'ANALY_UIO',
            'Anselm_MCORE',
            'BEIJING-CS-TH-1A_MCORE',
            'BEIJING-ERAII_MCORE',
            'BEIJING-ERA_MCORE',
            'BEIJING-PI_MCORE',
            'BEIJING-TIANJIN-TH-1A_MCORE',
            'BNL_KNL_MCORE',
            'CSCS-LCG2-HPC',
            'CSCS-LCG2-HPC_MCORE',
            'CSCS-LCG2-HPC_MCORE_TEST',
            'CSCS-TODI',
            'ES_ORNL_Titan',
            'HPC2N',
            'HPC2N_MCORE',
            'IN2P3-CC_HPC_DEBUG',
            'IN2P3-CC_HPC_IDRIS_MCORE',
            'IN2P3-CC_HPC_IDRIS_MCORE1',
            'IN2P3-CC_HPC_VIRTUAL',
            'LRZ-LMU_C2PAP',
            'LRZ-LMU_C2PAP_MCORE',
            'LRZ-LMU_MUC1_MCORE',
            'LRZ-LMU_MUC_MCORE',
            'LRZ-LMU_MUC_MCORE1',
            'MPPMU-DRACO_MCORE',
            'MPPMU-HYDRA_MCORE',
            'NERSC_Cori',
            'NERSC_Cori_2',
            'NERSC_Cori_p1_ES',
            'NERSC_Cori_p1_mcore',
            'NERSC_Cori_p2_ES',
            'NERSC_Cori_p2_mcore',
            'NERSC_Edison',
            'NERSC_Edison_ES',
            'NERSC_Edison_mcore',
            'NSC',
            'NSC_MCORE',
            'ORNL_Titan_MCORE',
            'ORNL_Titan_VIRTUAL',
            'RRC-KI-HPC2',
            'Titan_Harvester_MCORE',
            'Titan_Harvester_test_MCORE',
            'Titan_long_MCORE',
            'UIO',
            'UIO_ARC_TEST',
            'UIO_MCORE',
            'UIO_MCORE_LOPRI')

corepower_agis = {}

corepower_agis['AGLT2_LMEM']                      = 10.3503172086
corepower_agis['AGLT2_MCORE']                     = 10.3503172086
corepower_agis['AGLT2_SL6']                       = 10.3503172086
corepower_agis['AGLT2_SL7']                       = 10.3503172086
corepower_agis['ANALY_AGLT2_SL6']                 = 10.3503172086
corepower_agis['ANALY_AUSTRALIA']                 = 11.375
corepower_agis['ANALY_BEIJING']                   = 18.5810810811
corepower_agis['ANALY_BHAM_SL6']                  = 10.7297979798
corepower_agis['ANALY_BNL_LONG']                  = 9.98157248157
corepower_agis['ANALY_BNL_MCORE']                 = 9.98157248157
corepower_agis['ANALY_BNL_SHORT']                 = 9.98157248157
corepower_agis['ANALY_BNL_Test_2_CE_1']           = 9.98157248157
corepower_agis['ANALY_BU_ATLAS_Tier2_SL6']        = 20.0126760563
corepower_agis['ANALY_CERN']                      = 16.8891304348
corepower_agis['ANALY_CERN_CLOUD']                = 16.9415641476
corepower_agis['ANALY_CERN_HI']                   = 16.8891304348
corepower_agis['ANALY_CERN_T0']                   = 16.9415641476
corepower_agis['ANALY_CERN_T0_ART']               = 16.9415641476
corepower_agis['ANALY_CERN_T0_LSF']               = 16.9415641476
corepower_agis['ANALY_CERN_T0_SHORT']             = 16.9415641476
corepower_agis['ANALY_CONNECT']                   = 10.3821274298
corepower_agis['ANALY_CONNECT_SHORT']             = 10.3821274298
corepower_agis['ANALY_CPPM']                      = 10.879979571
corepower_agis['ANALY_CSCS']                      = 11.363857582
corepower_agis['ANALY_CSCS-HPC']                  = 11.363857582
corepower_agis['ANALY_CYF']                       = 14.1999671916
corepower_agis['ANALY_DCSC']                      = 14.2807723483
corepower_agis['ANALY_DESY-HH']                   = 11.2427469942
corepower_agis['ANALY_DESY-HH_UCORE']             = 11.2427469942
corepower_agis['ANALY_DESY-ZN']                   = 17.7699346405
corepower_agis['ANALY_ECDF_SL6']                  = 12.8996212121
corepower_agis['ANALY_ECDF_SL7']                  = 12.8996212121
corepower_agis['ANALY_FMPhI-UNIBA']               = 10.5133928571
corepower_agis['ANALY_FREIBURG']                  = 8.42007926024
corepower_agis['ANALY_FZK']                       = 12.559
corepower_agis['ANALY_FZK_UCORE']                 = 12.559
corepower_agis['ANALY_FZU']                       = 8.9726459758
corepower_agis['ANALY_GLASGOW_SL6']               = 8.740063593
corepower_agis['ANALY_GOEGRID']                   = 10.2658730159
corepower_agis['ANALY_GRIF-IRFU']                 = 10.6947402917
corepower_agis['ANALY_GRIF-LAL_HTCondor']         = 10.6947402917
corepower_agis['ANALY_GRIF-LPNHE']                = 10.6947402917
corepower_agis['ANALY_HPC2N']                     = 14.2807723483
corepower_agis['ANALY_HU_ATLAS_Tier2']            = 12.5894097222
corepower_agis['ANALY_IEPSAS-Kosice']             = 15.9359605911
corepower_agis['ANALY_IFAE']                      = 12.1205357143
corepower_agis['ANALY_IFIC']                      = 12.9677419355
corepower_agis['ANALY_IHEP']                      = 8.79993297587
corepower_agis['ANALY_IL-TAU-HEP-CREAM']          = 11.7509363296
corepower_agis['ANALY_IN2P3-CC']                  = 10.2628138236
corepower_agis['ANALY_INFN-FRASCATI']             = 9.11286919831
corepower_agis['ANALY_INFN-FRASCATI_PODtest']     = 9.11286919831
corepower_agis['ANALY_INFN-MILANO-ATLASC']        = 8.20586592179
corepower_agis['ANALY_INFN-NAPOLI']               = 8.5103427896
corepower_agis['ANALY_INFN-NAPOLI-RECAS']         = 8.5103427896
corepower_agis['ANALY_INFN-ROMA1']                = 9.33066239316
corepower_agis['ANALY_INFN-T1']                   = 10.91
corepower_agis['ANALY_JINR']                      = 12.7369680851
corepower_agis['ANALY_LANCS_SL6']                 = 14.4
corepower_agis['ANALY_LANCS_SL7']                 = 14.4
corepower_agis['ANALY_LAPP']                      = 11.2559633028
corepower_agis['ANALY_LIV_SL6']                   = 10.7333994054
corepower_agis['ANALY_LIV_SL7']                   = 10.7333994054
corepower_agis['ANALY_LPC']                       = 12.1723000824
corepower_agis['ANALY_LPSC']                      = 11.3995127893
corepower_agis['ANALY_LRZ']                       = 9.74263006294
corepower_agis['ANALY_LUNARC']                    = 0.812540822992
corepower_agis['ANALY_MANC_SL6']                  = 10.0954899777
corepower_agis['ANALY_MANC_SL7']                  = 10.0954899777
corepower_agis['ANALY_MCGILL']                    = 14.5
corepower_agis['ANALY_MCGILL_DOCKER']             = 14.5
corepower_agis['ANALY_MWT2_HIMEM']                = 10.3821274298
corepower_agis['ANALY_MWT2_HIMEM_MCORE']          = 10.3821274298
corepower_agis['ANALY_MWT2_MCORE']                = 10.3821274298
corepower_agis['ANALY_MWT2_SL6']                  = 10.3821274298
corepower_agis['ANALY_NCG-INGRID-PT_SL6']         = 8.01984126984
corepower_agis['ANALY_NIKHEF-ELPROD']             = 13.9
corepower_agis['ANALY_NIKHEF-ELPROD_SHORT']       = 13.9
corepower_agis['ANALY_NSC']                       = 14.2807723483
corepower_agis['ANALY_ORNL_Titan']                = 9.98157248157
corepower_agis['ANALY_OX_SL6']                    = 10.1547987616
corepower_agis['ANALY_PIC_SL6']                   = 12.1286846276
corepower_agis['ANALY_PSNC']                      = 24.0
corepower_agis['ANALY_QMUL_HIMEM_SL6']            = 11.0162429379
corepower_agis['ANALY_QMUL_SL6']                  = 11.0162429379
corepower_agis['ANALY_RAL_ECHO']                  = 10.0
corepower_agis['ANALY_RAL_MCORE']                 = 10.0
corepower_agis['ANALY_RALPP_SL6']                 = 10.0
corepower_agis['ANALY_RAL_SL6']                   = 10.0
corepower_agis['ANALY_RHUL_SL6']                  = 10.0188284519
corepower_agis['ANALY_ROMANIA02']                 = 9.60315533981
corepower_agis['ANALY_ROMANIA07']                 = 11.2523990181
corepower_agis['ANALY_RRC-KI']                    = 11.9201520913
corepower_agis['ANALY_RRC-KI-HPC']                = 15.5
corepower_agis['ANALY_RRC-KI-T1']                 = 15.5
corepower_agis['ANALY_SARA']                      = 17.13
corepower_agis['ANALY_SCINET']                    = 12.5
corepower_agis['ANALY_SFU_CONTAINER']             = 19.6358195834
corepower_agis['ANALY_SFU_T2']                    = 19.6358195834
corepower_agis['ANALY_SHEF_SL6']                  = 13.2
corepower_agis['ANALY_SiGNET']                    = 9.79996886675
corepower_agis['ANALY_SiGNET_DIRECT']             = 9.79996886675
corepower_agis['ANALY_SWT2_CPB']                  = 6.38582793164
corepower_agis['ANALY_T1_DE_KIT']                 = 11.4290129419
corepower_agis['ANALY_T1_ES_PIC']                 = 12.1286846276
corepower_agis['ANALY_T1_FR_CCIN2P3']             = 10.2628138236
corepower_agis['ANALY_T1_IT_CNAF']                = 10.91
corepower_agis['ANALY_T1_RU_JINR']                = 15.3199519231
corepower_agis['ANALY_T1_UK_RAL']                 = 10.0
corepower_agis['ANALY_T1_UK_RAL_ARC']             = 10.0
corepower_agis['ANALY_T2_AT_Vienna']              = 11.1
corepower_agis['ANALY_T2_BE_IIHE']                = 7.64801762115
corepower_agis['ANALY_T2_BE_UCL']                 = 9.2
corepower_agis['ANALY_T2_BR_SPRACE']              = 11.890625
corepower_agis['ANALY_T2_BR_UERJ']                = 10.00001
corepower_agis['ANALY_T2_CN_Beijing']             = 18.5810810811
corepower_agis['ANALY_T2_ES_IFCA']                = 11.4299496644
corepower_agis['ANALY_T2_FI_HIP']                 = 15.8005952381
corepower_agis['ANALY_T2_FR_IPHC']                = 12.567839196
corepower_agis['ANALY_T2_GR_Ioannina']            = 12.0
corepower_agis['ANALY_T2_HU_Budapest']            = 12.6298611111
corepower_agis['ANALY_T2_IN_TIFR']                = 10.0
corepower_agis['ANALY_T2_MY_UPM_BIRUNI']          = 10.00001
corepower_agis['ANALY_T2_PK_NCP']                 = 12.1469465649
corepower_agis['ANALY_T2_PL_Warsaw']              = 16.3999208861
corepower_agis['ANALY_T2_PT_NCG_Lisbon']          = 8.01984126984
corepower_agis['ANALY_T2_RU_IHEP']                = 8.79993297587
corepower_agis['ANALY_T2_RU_INR']                 = 13.5523255814
corepower_agis['ANALY_T2_RU_ITEP']                = 13.4051724138
corepower_agis['ANALY_T2_RU_JINR']                = 12.7369680851
corepower_agis['ANALY_T2_RU_PNPI']                = 11.0
corepower_agis['ANALY_T2_RU_RRC_KI']              = 11.9201520913
corepower_agis['ANALY_T2_TH_CUNSTDA']             = 10.00001
corepower_agis['ANALY_T2_TR_METU']                = 8.64880952381
corepower_agis['ANALY_T2_TW_Taiwan']              = 10.1398026316
corepower_agis['ANALY_T2_UK_London_Brunel']       = 11.95
corepower_agis['ANALY_T2_UK_SGrid_Bristol']       = 11.1695685087
corepower_agis['ANALY_T2_UK_SGrid_Bristol_long']  = 11.1695685087
corepower_agis['ANALY_T2_US_Florida']             = 16.5094339623
corepower_agis['ANALY_T2_US_Vanderbilt']          = 10.00001
corepower_agis['ANALY_T2_US_Wisconsin']           = 1.97072072072
corepower_agis['ANALY_TAIWAN_PNFS_SL6']           = 10.1398026316
corepower_agis['ANALY_TAIWAN_SL6']                = 10.1398026316
corepower_agis['ANALY_TAIWAN_XROOTD_SL6']         = 10.1398026316
corepower_agis['ANALY_TECHNION-HEP-CREAM']        = 11.75
corepower_agis['ANALY_TOKYO_ARC']                 = 18.1097005208
corepower_agis['ANALY_TR-10-ULAKBIM']             = 8.64930555556
corepower_agis['ANALY_TRIUMF']                    = 15.3388610763
corepower_agis['ANALY_TRIUMF_DOCKER']             = 15.3388610763
corepower_agis['ANALY_TRIUMF_HIMEM']              = 15.3388610763
corepower_agis['ANALY_TRIUMF_PPS']                = 15.3388610763
corepower_agis['ANALY_TRIUMF_SL7']                = 15.3388610763
corepower_agis['ANALY_UIO']                       = 14.2807723483
corepower_agis['ANALY_UNIBE-LHEP']                = 10.7127460169
corepower_agis['ANALY_UNIBE-LHEP-UBELIX']         = 10.7127460169
corepower_agis['ANALY_UTFSM']                     = 11.5651041667
corepower_agis['ANALY_VICTORIA']                  = 14.3604166667
corepower_agis['ANALY_WEIZMANN-CREAM']            = 11.75
corepower_agis['ANALY_wuppertalprod']             = 9.80005411255
corepower_agis['Anselm_MCORE']                    = 8.9726459758
corepower_agis['ARC-ES']                          = 14.2708539145
corepower_agis['Australia-ATLAS']                 = 11.375
corepower_agis['Australia-ATLAS_MCORE']           = 11.375
corepower_agis['BEIJING']                         = 18.5810810811
corepower_agis['BEIJING-CS-TH-1A_MCORE']          = 18.5810810811
corepower_agis['BEIJING-ERAII_MCORE']             = 18.5810810811
corepower_agis['BEIJING_MCORE']                   = 18.5810810811
corepower_agis['BEIJING-TIANJIN-TH-1A_MCORE']     = 18.5810810811
corepower_agis['BNL_ATLAS_2']                     = 9.98157248157
corepower_agis['BNL_PROD']                        = 9.98157248157
corepower_agis['BNL_PROD_MCORE']                  = 9.98157248157
corepower_agis['BNL_PROD_MCOREHIMEM']             = 9.98157248157
corepower_agis['BNL_Test_2_CE_1']                 = 9.98157248157
corepower_agis['BU_ATLAS_Tier2_MCORE']            = 20.0126760563
corepower_agis['BU_ATLAS_Tier2_SL6']              = 20.0126760563
corepower_agis['CA-MCGILL-CLUMEQ-T2']             = 14.5
corepower_agis['CA-MCGILL-CLUMEQ-T2_DOCKER']      = 14.5
corepower_agis['CA-MCGILL-CLUMEQ-T2_MCORE']       = 14.5
corepower_agis['CA-SCINET-T2']                    = 12.5
corepower_agis['CA-SCINET-T2_MCORE']              = 12.5
corepower_agis['CA-SFU-T2']                       = 19.6358195834
corepower_agis['CA-SFU-T2_CONTAINER']             = 19.6358195834
corepower_agis['CA-SFU-T2_CONTAINER_MCORE']       = 19.6358195834
corepower_agis['CA-SFU-T2_MCORE']                 = 19.6358195834
corepower_agis['CA-SFU-T2_MCORE_HIMEM']           = 19.6358195834
corepower_agis['CA-VICTORIA-WESTGRID-T2']         = 14.3604166667
corepower_agis['CA-VICTORIA-WESTGRID-T2_MCORE']   = 14.3604166667
corepower_agis['CERN-PROD']                       = 16.8891304348
corepower_agis['CERN-PROD_CLOUD']                 = 16.9415641476
corepower_agis['CERN-PROD_CLOUD_MCORE']           = 16.9415641476
corepower_agis['CERN-PROD_HI']                    = 16.8891304348
corepower_agis['CERN-PROD-preprod']               = 16.8891304348
corepower_agis['CERN-PROD-preprod_MCORE']         = 16.8891304348
corepower_agis['CERN-PROD_PUB_SCORE_SHORT']       = 16.8891304348
corepower_agis['CERN-PROD_T0_4MCORE']             = 16.88
corepower_agis['CERN-PROD_T0_MCORE']              = 16.88
corepower_agis['CERN-PROD_T0_SCORE']              = 16.88
corepower_agis['CERN-PROD_T0_SCORE_SHORT']        = 16.88
corepower_agis['CONNECT']                         = 10.3821274298
corepower_agis['CONNECT_BLUEWATERS']              = 10.3821274298
corepower_agis['CONNECT_BLUEWATERS_MCORE']        = 10.3821274298
corepower_agis['CONNECT_ES']                      = 10.3821274298
corepower_agis['CONNECT_ES_BLUEWATERS']           = 10.3821274298
corepower_agis['CONNECT_ES_BLUEWATERS_MCORE']     = 10.3821274298
corepower_agis['CONNECT_ES_FRESNOSTATE']          = 10.3821274298
corepower_agis['CONNECT_ES_ICC']                  = 10.3821274298
corepower_agis['CONNECT_ES_ICC_MCORE']            = 10.3821274298
corepower_agis['CONNECT_ES_JETSTREAM']            = 10.3821274298
corepower_agis['CONNECT_ES_JETSTREAM_MCORE']      = 10.3821274298
corepower_agis['CONNECT_ES_KARST']                = 10.3821274298
corepower_agis['CONNECT_ES_KARST_MCORE']          = 10.3821274298
corepower_agis['CONNECT_ES_MCORE']                = 10.3821274298
corepower_agis['CONNECT_ES_ODYSSEY']              = 10.3821274298
corepower_agis['CONNECT_ES_ODYSSEY_MCORE']        = 10.3821274298
corepower_agis['CONNECT_ES_UIUC']                 = 10.3821274298
corepower_agis['CONNECT_ES_UIUC_MCORE']           = 10.3821274298
corepower_agis['CONNECT_FRESNOSTATE']             = 10.3821274298
corepower_agis['CONNECT_JETSTREAM']               = 10.3821274298
corepower_agis['CONNECT_JETSTREAM_MCORE']         = 10.3821274298
corepower_agis['CONNECT_KARST']                   = 10.3821274298
corepower_agis['CONNECT_KARST_MCORE']             = 10.3821274298
corepower_agis['CONNECT_MCORE']                   = 10.3821274298
corepower_agis['CONNECT_MIDWAY']                  = 10.3821274298
corepower_agis['CONNECT_MIDWAY_MCORE']            = 10.3821274298
corepower_agis['CONNECT_ODYSSEY']                 = 10.3821274298
corepower_agis['CONNECT_ODYSSEY_MCORE']           = 10.3821274298
corepower_agis['CONNECT_RODEO']                   = 10.3821274298
corepower_agis['CONNECT_RODEO_MCORE']             = 10.3821274298
corepower_agis['CONNECT_UIUC']                    = 10.3821274298
corepower_agis['CONNECT_UIUC_MCORE']              = 10.3821274298
corepower_agis['CPPM']                            = 10.879979571
corepower_agis['CPPM_MCORE']                      = 10.879979571
corepower_agis['CSCS-LCG2']                       = 11.363857582
corepower_agis['CSCS-LCG2-HPC']                   = 11.363857582
corepower_agis['CSCS-LCG2-HPC_MCORE']             = 11.363857582
corepower_agis['CSCS-LCG2_MCORE']                 = 11.363857582
corepower_agis['CYFRONET-LCG2']                   = 14.1999671916
corepower_agis['CYFRONET-LCG2_MCORE']             = 14.1999671916
corepower_agis['DCSC']                            = 14.2807723483
corepower_agis['DCSC_MCORE']                      = 14.2807723483
corepower_agis['DESY-HH_MCORE']                   = 11.2427469942
corepower_agis['DESY-HH_UCORE']                   = 11.2427469942
corepower_agis['DESY-HH_UCORE']                   = 11.2248964803
corepower_agis['DESY-ZN']                         = 17.7699346405
corepower_agis['DESY-ZN_MCORE']                   = 17.7699346405
corepower_agis['EELA-UTFSM']                      = 11.5651041667
corepower_agis['EELA-UTFSM_MCORE']                = 11.5651041667
corepower_agis['ES_ORNL_Titan']                   = 9.98157248157
corepower_agis['FMPhI-UNIBA']                     = 10.5133928571
corepower_agis['FMPhI-UNIBA_MCORE']               = 10.5133928571
corepower_agis['FZK-LCG2']                        = 12.559
corepower_agis['FZK-LCG2_UCORE']                  = 12.559
corepower_agis['GoeGrid']                         = 10.2658730159
corepower_agis['GoeGrid_MCORE']                   = 10.2658730159
corepower_agis['GRIF-IRFU']                       = 10.6947402917
corepower_agis['GRIF-IRFU_MCORE']                 = 10.6947402917
corepower_agis['GRIF-LAL_HTCondor']               = 10.6947402917
corepower_agis['GRIF-LAL_HTCondor_MCORE']         = 10.6947402917
corepower_agis['GRIF-LPNHE']                      = 10.6947402917
corepower_agis['GRIF-LPNHE_MCORE']                = 10.6947402917
corepower_agis['HEPHY-UIBK']                      = 8.45769230769
corepower_agis['HPC2N']                           = 14.2807723483
corepower_agis['HPC2N_MCORE']                     = 14.2807723483
corepower_agis['HU_ATLAS_Tier2']                  = 12.5894097222
corepower_agis['HU_ATLAS_Tier2_MCORE']            = 12.5894097222
corepower_agis['IEPSAS-Kosice']                   = 15.9359605911
corepower_agis['IEPSAS-Kosice_MCORE']             = 15.9359605911
corepower_agis['IFAE']                            = 12.1205357143
corepower_agis['IFAE_MCORE']                      = 12.1205357143
corepower_agis['IFIC']                            = 12.9677419355
corepower_agis['IFIC_MCORE']                      = 12.9677419355
corepower_agis['IHEP_MCORE']                      = 8.79993297587
corepower_agis['IHEP_PROD']                       = 8.79993297587
corepower_agis['IL-TAU-HEP']                      = 11.7509363296
corepower_agis['IL-TAU-HEP_MCORE']                = 11.7509363296
corepower_agis['IN2P3-CC']                        = 10.2628138236
corepower_agis['IN2P3-CC_HIMEM']                  = 10.2628138236
corepower_agis['IN2P3-CC_HPC_IDRIS_MCORE1']       = 12.0
corepower_agis['IN2P3-CC_HPC_IDRIS_MCORE']        = 14.0
corepower_agis['IN2P3-CC_MCORE']                  = 10.2628138236
corepower_agis['IN2P3-CC_MCORE_HIMEM']            = 10.2628138236
corepower_agis['IN2P3-CC_VVL']                    = 10.2628138236
corepower_agis['IN2P3-LPSC']                      = 11.3995127893
corepower_agis['INFN-FRASCATI']                   = 9.11286919831
corepower_agis['INFN-FRASCATI_MCORE']             = 9.11286919831
corepower_agis['INFN-MILANO-ATLASC']              = 8.20586592179
corepower_agis['INFN-MILANO-ATLASC_MCORE']        = 8.20586592179
corepower_agis['INFN-NAPOLI-ATLAS']               = 8.5103427896
corepower_agis['INFN-NAPOLI-ATLAS_MCORE']         = 8.5103427896
corepower_agis['INFN-NAPOLI-RECAS']               = 8.5103427896
corepower_agis['INFN-NAPOLI-RECAS_MCORE']         = 8.5103427896
corepower_agis['INFN-NAPOLI-SCOPE']               = 8.5103427896
corepower_agis['INFN-ROMA1']                      = 9.33066239316
corepower_agis['INFN-ROMA1_MCORE']                = 9.33066239316
corepower_agis['INFN-T1']                         = 10.91
corepower_agis['INFN-T1-BA']                      = 10.91
corepower_agis['INFN-T1_HIMEM']                   = 10.91
corepower_agis['INFN-T1_MCORE']                   = 10.91
corepower_agis['ITEP_MCORE']                      = 13.4051724138
corepower_agis['ITEP_PROD']                       = 13.4051724138
corepower_agis['JINR_MCORE']                      = 12.7369680851
corepower_agis['JINR_PROD']                       = 12.7369680851
corepower_agis['LAPP']                            = 11.2559633028
corepower_agis['LAPP_MCORE']                      = 11.2559633028
corepower_agis['LPC']                             = 12.1723000824
corepower_agis['LPC_MCORE']                       = 12.1723000824
corepower_agis['LPSC_MCORE']                      = 11.3995127893
corepower_agis['LRZ-LMU_C2PAP_ES_MCORE']          = 9.74263006294
corepower_agis['LRZ-LMU_MCORE']                   = 9.74263006294
corepower_agis['LRZ-LMU_MUC_MCORE1']              = 8.0
corepower_agis['Lucille_CE']                      = 11.1164772727
corepower_agis['Lucille_MCORE']                   = 11.1164772727
corepower_agis['LUNARC']                          = 0.812540822992
corepower_agis['LUNARC_MCORE']                    = 0.812540822992
corepower_agis['MWT2_HIMEM']                      = 10.3821274298
corepower_agis['MWT2_HIMEM_MCORE']                = 10.3821274298
corepower_agis['MWT2_MCORE']                      = 10.3821274298
corepower_agis['MWT2_SL6']                        = 10.3821274298
corepower_agis['MWT2_VHIMEM']                     = 10.3821274298
corepower_agis['MWT2_VHIMEM_MCORE']               = 10.3821274298
corepower_agis['NCG-INGRID-PT_MCORE']             = 8.01984126984
corepower_agis['NCG-INGRID-PT_SL6']               = 8.01984126984
corepower_agis['NIKHEF-ELPROD']                   = 13.9
corepower_agis['NIKHEF-ELPROD-HighMem']           = 13.9
corepower_agis['NIKHEF-ELPROD_LONG']              = 13.9
corepower_agis['NIKHEF-ELPROD_MCORE']             = 13.9
corepower_agis['NSC']                             = 14.2807723483
corepower_agis['NSC_MCORE']                       = 14.2807723483
corepower_agis['OUHEP_OSG']                       = 12.921875
corepower_agis['OU_OCHEP_SWT2']                   = 12.921875
corepower_agis['OU_OSCER_ATLAS']                  = 13.8537234043
corepower_agis['OU_OSCER_ATLAS_MCORE']            = 13.8537234043
corepower_agis['OU_OSCER_ATLAS_OPP']              = 13.8537234043
corepower_agis['pic']                             = 12.1286846276
corepower_agis['pic_HIMEM']                       = 12.1286846276
corepower_agis['pic_HTCondor-mcore']              = 12.1286846276
corepower_agis['pic_HTCondor-score']              = 12.1286846276
corepower_agis['pic_IPV6']                        = 12.1286846276
corepower_agis['pic_MCORE']                       = 12.1286846276
corepower_agis['PNPI_PROD']                       = 11.0
corepower_agis['praguelcg2']                      = 9.10699740837
corepower_agis['praguelcg2_fzu_MCORE']            = 8.9726459758
corepower_agis['praguelcg2_fzu_SCORE']            = 8.9726459758
corepower_agis['praguelcg2_MCORE']                = 9.10699740837
corepower_agis['praguelcg2_IT4I_MCORE']           = 18.66
corepower_agis['PSNC']                            = 24.0
corepower_agis['PSNC_MCORE']                      = 24.0
corepower_agis['RAL-LCG2-ECHO']                   = 10.0
corepower_agis['RAL-LCG2-ECHO_MCORE']             = 10.0
corepower_agis['RAL-LCG2_ES']                     = 10.0
corepower_agis['RAL-LCG2_HIMEM_SL6']              = 10.0
corepower_agis['RAL-LCG2_SL6']                    = 10.0
corepower_agis['RAL-LCG2_UCORE']                  = 10.0
corepower_agis['RAL-LCG2_VAC']                    = 10.0
corepower_agis['RAL-LCG2_VHIMEM']                 = 10.0
corepower_agis['ROMANIA02']                       = 9.60315533981
corepower_agis['ROMANIA02_MCORE']                 = 9.60315533981
corepower_agis['ROMANIA07']                       = 11.2523990181
corepower_agis['ROMANIA07_ARC']                   = 11.2523990181
corepower_agis['ROMANIA07_MCORE']                 = 11.2523990181
corepower_agis['ROMANIA07_MCORE_ARC']             = 11.2523990181
corepower_agis['ROMANIA14']                       = 9.0
corepower_agis['ROMANIA14_MCORE']                 = 9.0
corepower_agis['ROMANIA16']                       = 9.0
corepower_agis['ROMANIA16_MCORE']                 = 9.0
corepower_agis['RRC-KI-HPC2']                     = 15.5
corepower_agis['RRC-KI_MCORE']                    = 11.9201520913
corepower_agis['RRC-KI_PROD']                     = 11.9201520913
corepower_agis['RRC-KI-T1']                       = 15.5
corepower_agis['RRC-KI-T1_MCORE']                 = 15.5
corepower_agis['SARA-MATRIX']                     = 17.13
corepower_agis['SARA-MATRIX_LONG']                = 17.13
corepower_agis['SARA-MATRIX_MCORE']               = 17.13
corepower_agis['SiGNET']                          = 9.79996886675
corepower_agis['SiGNET_MCORE']                    = 9.79996886675
corepower_agis['SiGNET-NSC_MCORE']                = 9.79996886675
corepower_agis['SWT2_CPB']                        = 6.38582793164
corepower_agis['SWT2_CPB_DEV_MCORE']              = 6.38582793164
corepower_agis['SWT2_CPB_DEV_SCORE']              = 6.38582793164
corepower_agis['SWT2_CPB_MCORE']                  = 6.38582793164
corepower_agis['Taiwan-LCG2']                     = 10.14
corepower_agis['Taiwan-LCG2_HIMEM']               = 10.14
corepower_agis['Taiwan-LCG2-HPC']                 = 10.14
corepower_agis['Taiwan-LCG2-HPC_HIMEM']           = 10.14
corepower_agis['Taiwan-LCG2-HPC_MCORE']           = 10.14
corepower_agis['Taiwan-LCG2-HPC_VL']              = 10.14
corepower_agis['Taiwan-LCG2_MCORE']               = 10.14
corepower_agis['Taiwan-LCG2_VL']                  = 10.14
corepower_agis['TECHNION-HEP']                    = 11.75
corepower_agis['TECHNION-HEP_MCORE']              = 11.75
corepower_agis['TOKYO_ARC']                       = 18.1097005208
corepower_agis['TOKYO_MCORE_ARC']                 = 18.1097005208
corepower_agis['TR-10-ULAKBIM']                   = 8.64930555556
corepower_agis['TR-10-ULAKBIM_MCORE']             = 8.64930555556
corepower_agis['TRIUMF']                          = 15.3388610763
corepower_agis['TRIUMF_DOCKER_UCORE']             = 15.3388610763
corepower_agis['TRIUMF_DOCKER_MCORE']             = 15.8019340974
corepower_agis['TRIUMF_HIMEM']                    = 15.3388610763
corepower_agis['TRIUMF_MCORE']                    = 15.3388610763
corepower_agis['TRIUMF_MCORE_LOMEM']              = 15.3388610763
corepower_agis['TRIUMF_PPS']                      = 15.3388610763
corepower_agis['TRIUMF_SL7']                      = 15.3388610763
corepower_agis['TRIUMF_SL7_MCORE']                = 15.3388610763
corepower_agis['TW-FTT_MCORE']                    = 10.1398026316
corepower_agis['TW-FTT_SL6']                      = 10.1398026316
corepower_agis['UIO']                             = 14.2807723483
corepower_agis['UIO_CLOUD']                       = 14.2807723483
corepower_agis['UIO_MCORE']                       = 14.2807723483
corepower_agis['UIO_MCORE_LOPRI']                 = 14.2807723483
corepower_agis['UKI-ARCHER_MCORE']                = 12.8996212121
corepower_agis['UKI-LT2-QMUL_ES']                 = 11.0162429379
corepower_agis['UKI-LT2-QMUL_HIMEM_SL6']          = 11.0162429379
corepower_agis['UKI-LT2-QMUL_MCORE']              = 11.0162429379
corepower_agis['UKI-LT2-QMUL_SL6']                = 11.0162429379
corepower_agis['UKI-LT2-RHUL_MCORE']              = 10.0188284519
corepower_agis['UKI-LT2-RHUL_SL6']                = 10.0188284519
corepower_agis['UKI-LT2-RHUL_VAC']                = 10.0229645094
corepower_agis['UKI-NORTHGRID-LANCS-HEP_ES']      = 14.4
corepower_agis['UKI-NORTHGRID-LANCS-HEP_MCORE']   = 14.4
corepower_agis['UKI-NORTHGRID-LANCS-HEP_SL7']     = 14.4
corepower_agis['UKI-NORTHGRID-LIV-HEP_MCORE']     = 10.7333994054
corepower_agis['UKI-NORTHGRID-LIV-HEP_MCORE_SL7'] = 10.7333994054
corepower_agis['UKI-NORTHGRID-LIV-HEP_SL6']       = 10.7333994054
corepower_agis['UKI-NORTHGRID-LIV-HEP_SL7']       = 10.7333994054
corepower_agis['UKI-NORTHGRID-LIV-HEP_VAC']       = 10.7338949455
corepower_agis['UKI-NORTHGRID-MAN-HEP_MCORE']     = 10.0954899777
corepower_agis['UKI-NORTHGRID-MAN-HEP_MCORE_SL7'] = 10.0954899777
corepower_agis['UKI-NORTHGRID-MAN-HEP_MCOREVAC']  = 10.0954899777
corepower_agis['UKI-NORTHGRID-MAN-HEP_SL6']       = 10.0954899777
corepower_agis['UKI-NORTHGRID-MAN-HEP_SL7']       = 10.0954899777
corepower_agis['UKI-NORTHGRID-MAN-HEP_VAC']       = 10.0954899777
corepower_agis['UKI-NORTHGRID-SHEF-HEP_MCORE']    = 13.2
corepower_agis['UKI-NORTHGRID-SHEF-HEP_SL6']      = 13.2
corepower_agis['UKI-SCOTGRID-ECDF_MCORE_SL7']     = 12.8996212121
corepower_agis['UKI-SCOTGRID-ECDF_SL7']           = 12.8996212121
corepower_agis['UKI-SCOTGRID-GLASGOW_MCORE']      = 8.740063593
corepower_agis['UKI-SCOTGRID-GLASGOW_MCOREVAC']   = 8.740063593
corepower_agis['UKI-SCOTGRID-GLASGOW_SL6']        = 8.740063593
corepower_agis['UKI-SCOTGRID-GLASGOW_VAC']        = 8.740063593
corepower_agis['UKI-SOUTHGRID-BHAM-HEP_MCOREVAC'] = 10.7297979798
corepower_agis['UKI-SOUTHGRID-BHAM-HEP_SL6']      = 10.7297979798
corepower_agis['UKI-SOUTHGRID-BHAM-HEP_VAC']      = 10.7297979798
corepower_agis['UKI-SOUTHGRID-CAM-HEP_VAC']       = 14.5588235294
corepower_agis['UKI-SOUTHGRID-OX-HEP_CLOUD']      = 10.1547987616
corepower_agis['UKI-SOUTHGRID-OX-HEP_MCORE']      = 10.1547987616
corepower_agis['UKI-SOUTHGRID-OX-HEP_SL6']        = 10.1547987616
corepower_agis['UKI-SOUTHGRID-OX-HEP_VAC']        = 10.1547987616
corepower_agis['UKI-SOUTHGRID-RALPP_MCORE']       = 10.0
corepower_agis['UKI-SOUTHGRID-RALPP_SL6']         = 10.0
corepower_agis['UNIBE-LHEP']                      = 10.7127460169
corepower_agis['UNIBE-LHEP_CLOUD']                = 10.7127460169
corepower_agis['UNIBE-LHEP_CLOUD_MCORE']          = 10.7127460169
corepower_agis['UNIBE-LHEP_MCORE']                = 10.7127460169
corepower_agis['UNIBE-LHEP-UBELIX']               = 10.7127460169
corepower_agis['UNIBE-LHEP-UBELIX_MCORE']         = 10.7127460169
corepower_agis['UNI-FREIBURG']                    = 8.42007926024
corepower_agis['UNI-FREIBURG_MCORE']              = 8.42007926024
corepower_agis['UTA_SWT2']                        = 7.54953145917
corepower_agis['UTA_SWT2_MCORE']                  = 7.54953145917
corepower_agis['WEIZMANN-LCG2']                   = 11.75
corepower_agis['WEIZMANN-LCG2_MCORE']             = 11.75
corepower_agis['wuppertalprod']                   = 9.80005411255
corepower_agis['wuppertalprod_HI']                = 9.80005411255
corepower_agis['wuppertalprod_MCORE']             = 9.80005411255
corepower_agis['wuppertalprod_UCORE']             = 9.80005411255

### select period

In [2]:
start_date = datetime.datetime(2017, 11, 1, 0, 0, 0)
#start_date = datetime.datetime(2018, 1, 30, 0, 0, 0)
curre_date = datetime.datetime(2018, 1, 31, 23, 59, 59)
#curre_date = datetime.datetime.utcnow()
days_of_data = (curre_date - start_date).days
timemargin = datetime.timedelta(1)

### selecting indices to scan

In [3]:
# get job archive indices
indices = es.cat.indices(index="jobs_archive_20*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
#for i in indices:
#    print(i)

selected_indices=[] 
for d in range(days_of_data+1):
    td = start_date + datetime.timedelta(d)
    ind = 'jobs_archive_'+td.strftime('%Y-%m-%d')
    if ind in indices:
        selected_indices.append(ind)

job_indices = ','.join(selected_indices)
print(job_indices)

jobs_archive_2017-11-01,jobs_archive_2017-11-02,jobs_archive_2017-11-03,jobs_archive_2017-11-04,jobs_archive_2017-11-05,jobs_archive_2017-11-06,jobs_archive_2017-11-07,jobs_archive_2017-11-08,jobs_archive_2017-11-09,jobs_archive_2017-11-10,jobs_archive_2017-11-11,jobs_archive_2017-11-12,jobs_archive_2017-11-13,jobs_archive_2017-11-14,jobs_archive_2017-11-15,jobs_archive_2017-11-16,jobs_archive_2017-11-17,jobs_archive_2017-11-18,jobs_archive_2017-11-19,jobs_archive_2017-11-20,jobs_archive_2017-11-21,jobs_archive_2017-11-22,jobs_archive_2017-11-23,jobs_archive_2017-11-24,jobs_archive_2017-11-25,jobs_archive_2017-11-26,jobs_archive_2017-11-27,jobs_archive_2017-11-28,jobs_archive_2017-11-29,jobs_archive_2017-11-30,jobs_archive_2017-12-01,jobs_archive_2017-12-02,jobs_archive_2017-12-03,jobs_archive_2017-12-04,jobs_archive_2017-12-05,jobs_archive_2017-12-06,jobs_archive_2017-12-07,jobs_archive_2017-12-08,jobs_archive_2017-12-09,jobs_archive_2017-12-10,jobs_archive_2017-12-11,jobs_archive_201

### running the scan

In [4]:
job_query = {
    "size": 0,
    "_source": [ "pandaid", "corecount", "cloud", "jobstatus", "processingtype", "actualcorecount",
                 "nevents", "cpuconsumptiontime",
                 "maxwalltime", "timeExe", "wall_time",
                 "hs06", "hs06sec", "computingsite", "cpu_eff"],
    'query':{
#        "match_all": {}
             'bool':{
                    'must':[
                         { "term": {"jobstatus": "finished" } },
#                         { "regexp": {"creationhost" : "hammercloud-ai*" } },
                         { "term": {"nevents" : "25" } },
                     ],
#                    'should':[
                       # { "term": {"prodsourcelabel": "rc_test" } },
                       # { "term": {"prodsourcelabel": "prod_test" } },
                       # { "term": {"prodsourcelabel": "managed" } }
#                        { "term": {"jobstatus": "finished" } },
#                        { "term": {"jobstatus": "failed" } },
#                    ]
             }
    }
                
}

scroll = scan(client=es, index=job_indices, query=job_query, scroll='1000000m', timeout="1000000m", size=10000)

count = 0
for res in scroll:
    count += 1
    
    #print(res)
    #if count>10000: break
    
    r = res['_source']
    pandaid = r['pandaid']
    cores = r['corecount']
    cloud = r['cloud']
    jobstatus = r['jobstatus']
    processingtype = r['processingtype']
    actualcorecount = r['actualcorecount']
    nevents  = r['nevents']
    cpuconsumptiontime  = r['cpuconsumptiontime']
    maxwalltime  = r['maxwalltime']
    timeExe  = r['timeExe']
    wall_time  = r['wall_time']
    hs06  = r['hs06']
    hs06sec  = r['hs06sec']
    computingsite  = r['computingsite']
    cpu_eff = r['cpu_eff']
    
    # if hs06 is None:
    #    continue
    
    if wall_time == 0:
        continue
    
    if actualcorecount != None and actualcorecount != 0:
        if hs06 is not None:
            hs06      /= actualcorecount
        wall_time *= actualcorecount
    elif cores != None and cores != 0:
        if hs06 is not None:
            hs06      /= cores
        wall_time *= cores
    
    # use corepower in AGIS
    if computingsite in corepower_agis:
        hs06 = corepower_agis[computingsite]
    else :
        hs06 = 0 
        
    # convert to cpu_time
    #wall_time *= cpu_eff
    
    if not count%100000: 
        print(count)
        #print(data)
    
    resource_type = 'grid'
    if computingsite in boincsites:
        resource_type = 'boinc'
    elif computingsite in cloudsites:
        resource_type = 'cloud'
    elif computingsite in hpcsites:
        resource_type = 'hpc'
        
    if resource_type == 'grid':
        if actualcorecount == 1:
            resource_type = 'grid1'
        elif actualcorecount == 8:
            resource_type = 'grid8'
    if processingtype != 'gangarobot-celt':
        continue
    jobs.append((pandaid, cores, cloud, jobstatus, processingtype, actualcorecount,
                 nevents, wall_time, hs06, hs06sec, resource_type, cpu_eff, computingsite))
    if count < 100:
        print(pandaid)
        print(jobs[-1])

#job=pd.DataFrame(data,columns=['initialjob', 'cores', 'errorcode', 'pandaid', 'failedattempt', 'child_ids', 'wall_time', 'jobstatus', 'processingtype'])
#del data
#job.head(100)

3732348564
(3732348564, 1, 'FR', 'finished', 'gangarobot-celt', 1, 25, 9014, 10.6947402917, None, 'grid1', 0.657089, 'GRIF-IRFU')
3762010349
(3762010349, 1, 'DE', 'finished', 'gangarobot-celt', 1, 25, 4181, 9.10699740837, None, 'grid1', 0.95886153, 'praguelcg2')
3801770274
(3801770274, 1, 'CA', 'finished', 'gangarobot-celt', 1, 25, 8795, 0, None, 'cloud', 0.95054007, 'IAAS')
3712282416
(3712282416, 1, 'DE', 'finished', 'gangarobot-celt', 1, 25, 5838, 9.80005411255, None, 'grid1', 0.9616307, 'wuppertalprod')
3772205406
(3772205406, 1, 'IT', 'finished', 'gangarobot-celt', 1, 25, 7029, 0, None, 'grid1', 0.96400625, 'ZA-WITS-CORE')
3720819397
(3720819397, 1, 'US', 'finished', 'gangarobot-celt', 1, 25, 6298, 7.54953145917, None, 'grid1', 0.9647507, 'UTA_SWT2')
3700078752
(3700078752, 1, 'FR', 'finished', 'gangarobot-celt', 1, 25, 5353, 11.2559633028, None, 'grid1', 0.9557258, 'LAPP')
3763006853
(3763006853, 1, 'DE', 'finished', 'gangarobot-celt', 1, 25, 4733, 10.2658730159, None, 'grid1', 0

### Print result

In [5]:
count = 0
types   = ('evgen', 'gangarobot-celt', 'simul', 'pile', 'deriv', 'merge', 'recon', 'pmerge', 'eventindex', 'skim', 'overlay')

resources = ('grid1', 'grid8', 'cloud', 'hpc')
data = {}

print('start')

for job in jobs:
    cores = job[1]
    cloud = job[2]
    jobstatus = job[3]
    processingtype = job[4]
    actualcorecount = job[5]
    nevents = job[6]
    wall_time = job[7]
    hs06 = job[8]
    resource_type = job[10]
    computingsite = job[12]

    count += 1
    
    if computingsite in hpcsites or computingsite in cloudsites or computingsite in boincsites:
        continue

    # avoid strange entries (different hs06 from AGIS) in simul
    #if cores != actualcorecount:
    #    continue
    
    # remove outliers
    if nevents / wall_time < 0.0001 or nevents / wall_time > 0.1:
        continue
        
    if resource_type in resources and processingtype in types:
        key = (resource_type, processingtype, hs06, computingsite)
        if (key not in data):
            data[key] = []
        data[key].append(nevents / wall_time)

for key,val in data.items():
    if len(val) > 2:
        print(key)
        print(statistics.median(val))
        print(statistics.mean(val))
        print(statistics.stdev(val))
        print(len(val))

start
('grid8', 'gangarobot-celt', 0, 'Australia-ATLAS_MCORE_TEST')
0.0017755962694399542
0.00176534922394449
8.751082463257893e-05
30
('grid1', 'gangarobot-celt', 0, 'LRZ-LMU_TEST')
0.0033715441672285905
0.0035294574433531716
0.0006788217794256876
43
('grid8', 'gangarobot-celt', 11.2523990181, 'ROMANIA07_MCORE_ARC')
0.0024940143655227454
0.0024142555601269376
0.0002737207501676573
5
('grid1', 'gangarobot-celt', 0, 'ANALY_UAM')
0.0036140837534116843
0.004242913103206336
0.0012738029651834023
16
('grid8', 'gangarobot-celt', 9.60315533981, 'ROMANIA02_MCORE')
0.002388415282353216
0.002074447143727708
0.0007444211652205035
28
('grid8', 'gangarobot-celt', 10.0188284519, 'UKI-LT2-RHUL_MCORE')
0.002207665213183129
0.00220123914043244
0.0003373419139154942
30
('grid8', 'gangarobot-celt', 8.79993297587, 'IHEP_MCORE')
0.0017857352772146873
0.0019937995190482233
0.0003356888174477915
28
('grid1', 'gangarobot-celt', 0, 'UKI-SCOTGRID-DURHAM_SL6')
0.0054800526085050415
0.0053082422495913184
0.000569